# Métricas

Para evaluar los dos modelos creados previamente primero nos centraremos en crear un conjunto de prueba siguiendo los siguientes pasos:

1. Obtener un conjunto de oraciones variadas (C)
    - Oraciones simples
    - Oraciones complejas 
    - Oraciones con ruido

2. TP - que recupere la query tal cual o lo más parecido
    - La query (c in C) si se encuentra en corpus
    - La query se encuentra parcialmente
       (tomamos oraciones del corpus y le metemos ruido)
       (tendríamos que revisar que el corpus si tenga la query parcial
       y luego revisar los resultados, si la busqueda da resultados
       pero en corpus no hay entonces es un falso positivo)


3. TN - oraciones donde ninguna palabra está dentro del corpus
    - Si regresa resultados es un falso negativo 

<p align="center">
  <img src="img/metricas.jpg" width="70%" alt="Metricas"/>
</p>

In [97]:
from os import listdir
from os.path import isfile, join
import json
from sklearn.model_selection import train_test_split #particiones
import string
import random
import pickle

Cargamos el corpus original para seleccionar algunas oraciones

In [101]:
def get_txt(path):
    """
    Regresa una lista con el contenido de todos los archivos de un directorio

    Args:
        path (str): ruta de la carpeta
    """
    text = []
    onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
    
    for file in onlyfiles:
        with open(path+"/"+file, 'rb') as f:
            text.append(json.loads(f.read().decode('utf-8', 'replace')))
    return text

# Guardamos los subtitulos de todos los canales
videos_original = get_txt("../corpus/data")#Nota: no print de todo
# Obtenemos el conjunto de test
_, test = train_test_split(videos_original, test_size=0.05) #5%
_, test = train_test_split(videos_original, test_size=0.05) #2.5%

In [102]:
def get_subtitles():
    '''
    Regresa los subtitulos del corpus
    '''
    test_subtitles = []
    for chanel in test:
        for video in chanel:
            if 'subtitles' in video:
                for s in video['subtitles']:
                    test_subtitles.append(s['text'])
    return test_subtitles

Del 2.5% de oraciones tomadas del corpus tomamos la mitad para meterles ruido

In [103]:
test_TP, test_TPR = train_test_split(get_subtitles(), test_size=0.5)
print(test_TPR[:10], len(test_TPR), len(test_TP))

['soltar imágenes textos vídeos algunos', 'que exista otra vida además de la', 'células básicamente una célula es un', 'ejercicios para modificar el atributo de', 'infinito por no hablar de que los', 'de perclorato muy tóxicas la exposición', 'extremos ya sea un agujero de gusano', 'para entrar en la fase 2 en la segunda', 'infectar otras células y limpia el campo', 'nuestros vídeos ya nos ayuda un montón'] 4744 4743


In [104]:
def insert(s, num):
    """
    Agrega n cadenas aleatorias a una lista

    Args:
        s (list): lista de cadenas
        num (int): número de cadenas aleatorias a insertar
    """
    for _ in range(num):
        noise = ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(6))
        ix = random.randint(0,len(s)-1)
        s.insert(ix, noise)

def make_noise():
    """
    A cada oración del conjunto le agrega n cadenas aleatorias
    """
    for i, sentence in enumerate(test_TPR):
        s = sentence.split(" ")
        insert(s, 2)
        test_TPR[i] = ' '.join(s)

make_noise()
print(test_TPR[:10], len(test_TPR))

['soltar imágenes textos UOFoHI uso0Ob vídeos algunos', 'que exista otra Wl8dY8 vida además YIa9NP de la', 'SzxpRZ células básicamente una célula es 518zy7 un', 'ejercicios para N2LnNW modificar el atributo 4XCQtI de', 'infinito por no hablar de que Y4hQyV ZJBZXP los', 'Zpt6MU de perclorato VEAjM5 muy tóxicas la exposición', 'extremos ya sea y8qvc9 un agujero de zI6viV gusano', 'para entrar en la fase oYWMw0 2 en fMHaR0 la segunda', 'infectar otras células y limpia m1LEvD NYQxll el campo', 'nuestros vídeos ya nos ayuda VSKiqw jSNeyb un montón'] 4744


Generamos 1.25% de oraciones con palabras que no están en el corpus de tal manera que el conjunto de prueba queda compuesto por:
- 1.25% oraciones que se encuentran tal cual en el corpus
- 1.25% oraciones que se encuentran  en el corpus pero con ruido
- 0.625% oraciones que no comparten palabras con el corpus

In [105]:
def make_sentence():
    """
    Crea un conjunto de cadenas aleatorias
    """
    sentence = []
    lg = [3,5,10,20]
    for _ in range(len(test_TP)//2):
        s = [""]
        insert(s, lg[random.randint(0,len(lg)-1)])
        sentence.append(' '.join(s))
    return sentence
    
test_TN = make_sentence()
print(test_TN[:10], len(test_TN))

['pNjqoS emchMs dfM1Ef cxBJh7 RJDbTr y29Kn0 TxCGAa CqrMvb wQ99Lc PzKkSW ', 'vn46SG JrrVQM POFCX6 tiDUMa IxY0gV ', 'jhk88X mDoQZI 02xtuz ', 'uxJ5jc LqOCSj nSODbT wMv8NX W57xf8 ', 'Zh25XY 10b3Ew vtRuqN 7XNZSk Iacqgg LMt9vE 6d4KKh h4oAvA PM4UQM rIBzIt ', 'wxwJ2d bRe5iw M2XQ9d U6gv5t OrW0UT HxEbhJ AJX6Mn 3WyVWI zYlL8Q FyCOb1 xXOvr2 fp36CV 4rZvOy wIojqL dRzvEp p1E7C0 brW1iH STI08e li93hH 80O0c0 ', 'Aurwut 7e9Kd2 rq6LEH sPkDEq Tfsy5t Uz5Bkb WqN0me jxSqK4 VxBWoF K1qTof THIyVL 0RVwOo c7J6JC eV6eRw i5XVXf lpN2gx xset6Y ehjgzc dcP6si 6PKxXu ', 'iLbyl4 xllOvL 7Tiz6k 83XYe0 ONmETg WTW9iR Vga8Zx x9qcqP plZzgC wWbxUH ', 'glIhY9 pxsht8 POTFfg ', 'W0QxIW gC2XRE n78bdo 9YUiCY lQACY5 '] 2371


Guardamos el conjunto de prueba

In [106]:
test_all = test_TP + test_TPR + test_TN
pickle.dump(test_all, open("../pkl/test.pkl", "wb"))

In [107]:
with open("../pkl/test.pkl", "rb") as f:
    test_all = pickle.load(f)

test_all[4740:4750]

['constantemente juntas todas esas',
 'ciego y la evaluación científica para',
 'el humano moderno surgió al menos 200',
 'soltar imágenes textos UOFoHI uso0Ob vídeos algunos',
 'que exista otra Wl8dY8 vida además YIa9NP de la',
 'SzxpRZ células básicamente una célula es 518zy7 un',
 'ejercicios para N2LnNW modificar el atributo 4XCQtI de',
 'infinito por no hablar de que Y4hQyV ZJBZXP los',
 'Zpt6MU de perclorato VEAjM5 muy tóxicas la exposición',
 'extremos ya sea y8qvc9 un agujero de zI6viV gusano']

In [108]:
len(test_all)

11858

test[0] = "holaa"

resp[0] = "subtítulo encontrado dada la query test[0]"

guardas en un pickle resp